In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.output_subarea output_text output_stream output_stdout{width:97%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.text_cell_render.rendered_html{font-size:14pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:14px;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [3]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
#    separators=["\n\n","\n"," ",""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

193

In [3]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()   #안하면 403에러
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [6]:
len(embedding.embed_query("소득세법"))

4096

In [8]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# # 데이터를 처음 업로드 할 때 
# index_name = "tax-index-upstage"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )  ##한번만 실행
# # 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# # 업로드한 벡터DB를 가져올때
database = PineconeVectorStore(
    embedding=embedding,  # 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 15.6 ms
Wall time: 999 μs


# 2. 답변 생성을 위한 Retrieval

In [9]:
query = "연봉이 5천만원인 직장인의  소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3)  # 기본 k값:4

In [21]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

'[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1., 2025. 12. 23.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택

In [10]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrieved_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. 답변 생성

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [12]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" # 느리지만 더 깊게 추론함(low, medium)
)

In [13]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
[context]를 참고해서 사용자의 질문에 답변해주세요
[context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [14]:
ai_message = llm.invoke(prompt)

In [15]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세 계산은 다음과 같은 단계로 진행됩니다. 단, **자녀세액공제 및 기타 추가 공제는 적용되지 않는다고 가정**하고, **2024년 기준 현행 법령**을 적용합니다.

---

### **1. 근로소득공제 계산 (제47조)**
- **총급여액**: 5,000만원  
- **근로소득공제율** (2024년 기준):  
  - 1,500만원 이하: 70%  
  - 1,500만원 초과 ~ 4,500만원 이하: 40%  
  - 4,500만원 초과 ~ 1억원 이하: 15%  
  - 1억원 초과: 10%  

**계산**:  
- 1,500만원 × 70% = **1,050만원**  
- (4,500만원 - 1,500만원) × 40% = **1,200만원**  
- (5,000만원 - 4,500만원) × 15% = **75만원**  
- **총 근로소득공제액** = 1,050 + 1,200 + 75 = **2,325만원**  
  (단, 최대 2,000만원 한도 적용 → **2,000만원** 공제)

---

### **2. 과세표준 계산**
- **과세표준** = 총급여액 - 근로소득공제  
  = 5,000만원 - 2,000만원 = **3,000만원**

---

### **3. 종합소득산출세액 계산 (누진세율 적용)**
2024년 종합소득세 세율 (과세표준 기준):  
| 과세표준 구간 | 세율 | 누진공제 |
|---------------|------|----------|
| 1,200만원 이하 | 6% | - |
| 1,200만원 초과 ~ 4,600만원 이하 | 15% | 108만원 |
| 4,600만원 초과 ~ 8,800만원 이하 | 24% | 522만원 |

**계산**:  
- 3,000만원 × 15% - 108만원 = **450만원 - 108만원 = 342만원**

---

### **4. 근로소득세액공제 적용 (제59조)**
- **총급여액 5,000만원**은 **3,300만원 초과 ~ 7,000만원 이하** 구간에 해당합

# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase를 활용한 RAG구현(UpstageEmbedding) 참조

In [ ]:
rag_chain = promptTemplate | llm | output_parser
rag_chain.invoke({'context':retrieved_doc,'question':query})

# langchain 전달

In [19]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model="gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")
                  
embedding = UpstageEmbeddings( model="solar-embedding-1-large-passage")

# 2. 업로드한 벡터db 가져올 때
vectorstore = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# vectorstore.similarity_search("질문", k=2) - 어떤 질문이 들어올지 몰라서 쓸수 없다.

# 4.프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return "\n\n---\n\n".join([doc.page_content for doc in documents])

In [20]:
# 6. RAG 체인 구성(LCEL 방식)
from langchain_core.runnables import RunnablePassthrough  # {"query":"~"}  => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough()  # 질문 그대로 전달
    }
    | prompt  # prompt에 context와 query 변수 주입
    | llm     # llm에 
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'연봉 5천만원인 직장인의 소득세는 근로소득공제, 세율, 세액공제 등을 종합적으로 계산해야 하며, 제공된 문맥만으로는 정확한 세액을 산출할 수 없습니다. 추가 정보(가족 구성, 연금계좌 납입액, 자녀 세액공제 등)가 필요합니다. \n\n간단히 설명드리면, 총급여 5,500만원일 경우 근로소득공제 후 과세표준은 약 3,850만원이며, 기본세율(6~24%) 적용 후 세액공제를 차감해야 합니다. 정확한 계산을 위해서는 세무사나 국세청 계산기를 활용하시기 바랍니다.'